## Run Grid Search on Machine Learning Algorithms

In [ ]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
import datetime

In [7]:
%run ml_helpers.ipynb

In [ ]:
#Import bucketized data:
#data = ml_helpers.convert_to_categorical(data, [columns_to_convert])